In [201]:
from pathlib import Path
import sgkit as sg
import anndata as ad
import pandas as pd
import scanpy as sc

/Users/jan.engelmann/mambaforge/envs/scgeno/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/Users/jan.engelmann/mambaforge/envs/scgeno/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/Users/jan.engelmann/mambaforge/envs/scgeno/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/Users/jan.engelmann/mambaforge/envs/scgeno/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/Users/jan.engelmann/mambaforge/envs/scgeno/lib/python3.10/site-packag

In [ ]:
DATA = Path("/Users/jan.engelmann/projects/sc-eqtl/data")

In [6]:
vcf_file_path = DATA / "OneK1K_imputation_post_qc_r2_08/filter_vcf_r08/chr22.dose.filtered.R2_0.8.vcf.gz"

zarr_path = vcf_file_path.parent.parent / "filter_zarr_r08"
zarr_path.mkdir(exist_ok=True)

In [7]:
icf_file_path = zarr_path / vcf_file_path.with_suffix(".icf").name
zarr_file_path = (zarr_path / vcf_file_path.stem).with_suffix(".vcz")
zarr_file_path

PosixPath('/Users/jan.engelmann/projects/sc-eqtl/data/OneK1K_imputation_post_qc_r2_08/filter_zarr_r08/chr22.dose.filtered.R2_0.8.vcz')

In [10]:
! vcf2zarr explode {vcf_file_path} {icf_file_path}

    Scan: 100%|███████████████████████████| 1.00/1.00 [00:00<00:00, 1.27files/s]
 Explode: 100%|███████████████████████████| 143k/143k [01:49<00:00, 1.30kvars/s]


In [18]:
! vcf2zarr encode {icf_file_path} {zarr_file_path}

  Encode: 100%|████████████████████████████| 3.11G/3.11G [01:07<00:00, 46.3MB/s]
Finalise: 100%|████████████████████████████| 16.0/16.0 [00:00<00:00, 268array/s]


In [19]:
! vcf2zarr inspect {zarr_file_path}

name                   dtype    stored      size           ratio    nchunks  chunk_size    avg_chunk_stored    shape              chunk_shape       compressor                                                      filters
---------------------  -------  ----------  ----------  --------  ---------  ------------  ------------------  -----------------  ----------------  --------------------------------------------------------------  ------------
/call_GP               float32  105.02 MiB  1.65 GiB      16             30  56.44 MiB     3.5 MiB             (143083, 1034, 3)  (10000, 1000, 3)  Blosc(cname='zstd', clevel=7, shuffle=NOSHUFFLE, blocksize=0)   None
/call_DS               float32  69.89 MiB   564.38 MiB     8.1           30  18.81 MiB     2.33 MiB            (143083, 1034)     (10000, 1000)     Blosc(cname='zstd', clevel=7, shuffle=NOSHUFFLE, blocksize=0)   None
/call_genotype         int8     5.26 MiB    282.19 MiB    54             30  9.41 MiB      179.58 KiB          (143083, 1

In [249]:
scdata_path = DATA / "debug_OneK1K_cohort_gene_expression_matrix_14_celltypes.h5ad"
scdata = ad.read_h5ad(scdata_path)
scdata

AnnData object with n_obs × n_vars = 25908 × 32738
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'pool', 'individual', 'percent.mt', 'latent', 'nCount_SCT', 'nFeature_SCT', 'cell_type', 'cell_label', 'sex', 'age'
    var: 'GeneSymbol', 'features'

In [250]:
gdata = sg.load_dataset(zarr_file_path)
del gdata.attrs["vcf_header"]
gdata

<xarray.Dataset> Size: 3GB
Dimensions:               (variants: 143083, samples: 1034, FORMAT_GP_dim: 3,
                           ploidy: 2, contigs: 1, filters: 3, alleles: 2)
Dimensions without coordinates: variants, samples, FORMAT_GP_dim, ploidy,
                                contigs, filters, alleles
Data variables: (12/19)
    call_DS               (variants, samples) float32 592MB dask.array<chunksize=(10000, 1000), meta=np.ndarray>
    call_GP               (variants, samples, FORMAT_GP_dim) float32 2GB dask.array<chunksize=(10000, 1000, 3), meta=np.ndarray>
    call_genotype         (variants, samples, ploidy) int8 296MB dask.array<chunksize=(10000, 1000, 2), meta=np.ndarray>
    call_genotype_mask    (variants, samples, ploidy) bool 296MB dask.array<chunksize=(10000, 1000, 2), meta=np.ndarray>
    call_genotype_phased  (variants, samples) bool 148MB dask.array<chunksize=(10000, 1000), meta=np.ndarray>
    contig_id             (contigs) object 8B dask.array<chunksize=(1,), meta=np.ndarray>
    ...                    ...
    variant_contig        (variants) int8 143kB dask.array<chunksize=(10000,), meta=np.ndarray>
    variant_filter        (variants, filters) bool 429kB dask.array<chunksize=(10000, 3), meta=np.ndarray>
    variant_id            (variants) object 1MB dask.array<chunksize=(10000,), meta=np.ndarray>
    variant_id_mask       (variants) bool 143kB dask.array<chunksize=(10000,), meta=np.ndarray>
    variant_position      (variants) int32 572kB dask.array<chunksize=(10000,), meta=np.ndarray>
    variant_quality       (variants) float32 572kB dask.array<chunksize=(10000,), meta=np.ndarray>
Attributes:
    source:            bio2zarr-0.1.1
    vcf_zarr_version:  0.2

Ideas
- load genetic data into anndata
- use xarray
- have joint datastructure with subsetting

In [251]:
import logging

logger = logging.getLogger(__name__)


def match_samples(gdata, scdata, g_on="sample_id", sc_on="individual", verbose=False):
    """Match samples between genetic and single-cell data."""
    scdata = scdata[scdata.obs[sc_on].sort_values().index]

    sc_index = pd.Index(scdata.obs[sc_on].unique())
    g_index = pd.Index(gdata[g_on].to_series())
    keep_samples = sc_index.intersection(g_index)
    if verbose:
        all_samples = sc_index.union(g_index)
        logger.warning("Keeping %s/%s samples", len(keep_samples), len(all_samples))
        logger.warning("Dropping %s/%s samples from genetic data", len(g_index) - len(keep_samples), len(g_index))
        logger.warning("Dropping %s/%s samples from single-cell data", len(sc_index) - len(keep_samples), len(sc_index))

    gdata = gdata.assign_coords(samples=gdata[g_on])
    gdata = gdata.sel(samples=keep_samples)
    scdata = scdata[scdata.obs[sc_on].isin(keep_samples)]

    return gdata, scdata


gdata, scdata = match_samples(gdata, scdata, verbose=True)

Keeping 20/1034 samples
Dropping 1014/1034 samples from genetic data
Dropping 0/20 samples from single-cell data


In [252]:
annot = (
    sc.queries.biomart_annotations(
        "hsapiens",
        ["ensembl_gene_id", "start_position", "end_position", "chromosome_name"],
    )
    .set_index("ensembl_gene_id")
    .drop_duplicates()
)
annot

,start_position,end_position,chromosome_name
ensembl_gene_id,,,
ENSG00000210049,577,647,MT
ENSG00000211459,648,1601,MT
ENSG00000210077,1602,1670,MT
ENSG00000210082,1671,3229,MT
ENSG00000209082,3230,3304,MT
...,...,...,...
ENSG00000232679,221819842,222064773,1
ENSG00000200033,221837334,221837437,1
ENSG00000228437,221966341,221984964,1


In [253]:
scdata = scdata[:, scdata.var.index.isin(annot.index)]
scdata.var["chrom"] = annot.loc[scdata.var.index, "chromosome_name"].values
scdata.var["start"] = annot.loc[scdata.var.index, "start_position"].values
scdata.var["end"] = annot.loc[scdata.var.index, "end_position"].values
scdata.var

/var/folders/hm/5_qpdtz57vv91d3bdgqdbqd82hn_ys/T/ipykernel_69799/3317223926.py:2: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  scdata.var["chrom"] = annot.loc[scdata.var.index, "chromosome_name"].values


,GeneSymbol,features,chrom,start,end
Geneid,,,,,
ENSG00000243485,MIR1302-10,MIR1302-10,1,29554,31109
ENSG00000237613,FAM138A,FAM138A,1,34554,36081
ENSG00000186092,OR4F5,OR4F5,1,65419,71585
ENSG00000238009,RP11-34P13.7,RP11-34P13.7,1,89295,133566
ENSG00000239945,RP11-34P13.8,RP11-34P13.8,1,89551,91105
...,...,...,...,...,...
ENSG00000212907,MT-ND4L,MT-ND4L,MT,10470,10766
ENSG00000198886,MT-ND4,MT-ND4,MT,10760,12137
ENSG00000198786,MT-ND5,MT-ND5,MT,12337,14148


In [254]:
def match_regions(scdata, gdata, sc_on="chrom", verbose=False):
    """Filter SNPs and genes to same chromosome"""
    idx_to_chrom = gdata.contig_id.to_series()  # idx, chrom
    sc_regions = set(scdata.var[sc_on].unique())
    keep_regions = sc_regions.intersection(idx_to_chrom.values)

    keep_genes = scdata.var[sc_on].isin(keep_regions)

    keep_snps = gdata.variant_contig.isin(idx_to_chrom[idx_to_chrom.isin(keep_regions)].index).data.compute()
    if verbose:
        logger.warning("Keeping %s/%s genes", keep_genes.sum(), len(keep_genes))
        logger.warning("Keeping %s/%s snps", keep_snps.sum(), len(keep_snps))

    scdata = scdata[:, keep_genes]
    gdata = gdata.isel(variants=keep_snps)
    return scdata, gdata


scdata, gdata = match_regions(scdata, gdata, verbose=True)

Keeping 666/30774 genes
Keeping 143083/143083 snps
